<a href="https://colab.research.google.com/github/ahekmati/ahekmati.github.io/blob/master/OilLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crude Oil Price Forcasting

# Requirements
    anaconda                  2019.10
    python                    3.7.4
    keras                     2.1.5
    tensorflow                2.0.0
    Ploty                     4.2.1
    Dataset                   DCOILWTICO.csv
    jupyter notebook

# Import Libraries

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Read Dataset

In [7]:
data=pd.read_csv("DCOILWTICO.csv")
data.tail()

,DATE,DCOILWTICO
8933,2020-03-31,20.51
8934,2020-04-01,20.28
8935,2020-04-02,25.18
8936,2020-04-03,28.36
8937,2020-04-06,26.21


In [0]:
## forcasting for 30 days
pre_look = 30

# Dataset Cleaning 

In [0]:
data['DCOILWTICO'].replace('.', np.nan, inplace=True)
data.dropna(subset=['DCOILWTICO'], inplace=True)

# Normalization and Transformation

In [0]:
price_value = data['DCOILWTICO'].values
price_value= price_value.astype('float32')
price_value = np.reshape(price_value, (-1, 1))
Norm = MinMaxScaler(feature_range=(0, 1))
price_value= Norm.fit_transform(price_value)

# Splitting

In [0]:
splitting = 0.75
cut = int(splitting*len(price_value))
train_data = price_value[:cut]
test_data = price_value[cut:]
date_cut_1 = data['DATE'][:cut]
date_cut_2 = data['DATE'][cut:]

# Time Series Generator

In [0]:
tr_g = TimeseriesGenerator(train_data, train_data, length=pre_look, batch_size=32)     
tes_g = TimeseriesGenerator(test_data, test_data, length=pre_look, batch_size=1)

# LSTM(Long Short Term Memory) Model

In [0]:
model = Sequential()
model.add(
    LSTM(20,
        activation='relu',
        input_shape=(pre_look,1))
)
model.add(Dense(1))
model.compile(optimizer='SGD', loss='mean_squared_error')

# Training

In [14]:
model.fit_generator(tr_g, epochs=70, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/70
202/202 [==============================] - 2s 10ms/step - loss: 0.0194
Epoch 2/70
202/202 [==============================] - 2s 11ms/step - loss: 0.0105
Epoch 3/70
202/202 [==============================] - 2s 10ms/step - loss: 0.0044
Epoch 4/70
202/202 [==============================] - 2s 11ms/step - loss: 0.0013
Epoch 5/70
202/202 [==============================] - 2s 11ms/step - loss: 3.5519e-04
Epoch 6/70
202/202 [==============================] - 2s 11ms/step - loss: 2.5367e-04
Epoch 7/70
202/202 [==============================] - 2s 11ms/step - loss: 2.4300e-04
Epoch 8/70
202/202 [==============================] - 2s 10ms/step - loss: 2.3928e-04
Epoch 9/70
202/202 [==============================] - 2s 10ms/step - loss: 2.4136e-04
Epoch 10/70
202/202 [==============================] - 2s 10ms/step - loss: 2.3983e-04
Epoch 11/70
202/202 [==============================] - 2s 11ms/step - loss: 2.3

# Testing

In [15]:
p = model.predict_generator(tes_g)

Instructions for updating:
Please use Model.predict, which supports generators.


# Rescaling

In [0]:
## prediction
predictiont  = Norm.inverse_transform(p)
## train data
a= train_data.reshape((-1,1))
price_t= Norm.inverse_transform(a)
## test data
aa= test_data.reshape((-1,1))
price_tt= Norm.inverse_transform(aa)
## create Datsframe
pre=pd.DataFrame(predictiont)
pre1=pd.DataFrame(price_t)
pre11=pd.DataFrame(price_tt)

# Visualization of Testing Accuracy

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(  x = date_cut_1,
    y = pre1[0],
                    mode='lines',
                    name='Training',line_color='rgb(56,176,2)'))
fig.add_trace(go.Scatter(     x = date_cut_2,
    y = pre[0],
                    mode='lines',
                    name='prediction',line_color='red'))
fig.add_trace(go.Scatter(        x = date_cut_2,
    y = pre11[0],
                    mode='lines',
                    name='Testing',line_color='rgb(2,40,50)'))

fig.update_layout(title='Crude Oil Price Forcasting',
                   xaxis_title='Date',
                   yaxis_title='Price')

fig.show()

# Forecasting

In [0]:
def ForCast_price_Days(forcast_days_length, model):
    for_list = price_value[-pre_look:]
    
    for _ in range(forcast_days_length):
        x = for_list[-pre_look:]
        x = x.reshape((1, pre_look, 1))
        out = model.predict(x)[0][0]
        for_list = np.append(for_list, out)
    for_list = for_list[pre_look-1:]
    pre_date = data['DATE'].values[-1]
    forcast_dates = pd.date_range(pre_date, periods=forcast_days_length+1).tolist()
        
    return for_list,forcast_dates

In [0]:
## forcast 30 Days
price_30,Days_30 = ForCast_price_Days(30, model)
price_30= price_30.reshape((-1,1))

## forcast price
price_30=Norm.inverse_transform(price_30)
f=pd.DataFrame(price_30)
## whole Dataset
whole_datat= price_value.reshape((-1,1))
whole_datat= Norm.inverse_transform(whole_datat)
whole_datat=pd.DataFrame(whole_datat)

# Next 30 Days Price Forcasting

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter( x = data['DATE'].tolist(),
    y = whole_datat[0],
                    mode='lines',
                    name='Past_Price',line_color='rgb(55,30,50)'))
fig.add_trace(go.Scatter( x = Days_30,
    y = f[0],
                    mode='lines',
                    name='Forcast_Price_30',line_color='red'))
fig.update_layout(title='Crude Oil Price Forcasting',
                   xaxis_title='Date',
                   yaxis_title='Price')

fig.show()

In [35]:
print(price_30)

[[26.209997]
 [23.464659]
 [23.418488]
 [23.380024]
 [23.34145 ]
 [23.303612]
 [23.261744]
 [23.217539]
 [23.172268]
 [23.12658 ]
 [23.085436]
 [23.055264]
 [23.021072]
 [22.988045]
 [22.9566  ]
 [22.924923]
 [22.896328]
 [22.870111]
 [22.851574]
 [22.82813 ]
 [22.81138 ]
 [22.790936]
 [22.773306]
 [22.756586]
 [22.744646]
 [22.733883]
 [22.724005]
 [22.708166]
 [22.692606]
 [22.671375]
 [22.646227]]
